In [2]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

CSV_FILE = 'landmarks.csv'
assert os.path.exists(CSV_FILE), f"CSV not found: {CSV_FILE}"

df = pd.read_csv(CSV_FILE)
print('rows,cols:', df.shape)

df.head()

rows,cols: (40, 1567)


,label,face_0_x,face_0_y,face_0_z,face_1_x,face_1_y,face_1_z,face_2_x,face_2_y,face_2_z,...,pose_30_z,pose_30_v,pose_31_x,pose_31_y,pose_31_z,pose_31_v,pose_32_x,pose_32_y,pose_32_z,pose_32_v
0,0,-0.055945,0.265708,-0.037593,0.0,0.0,-0.077665,-0.026032,0.066378,-0.038437,...,1.230152,0.000044,-0.004757,13.493138,0.663363,0.000173,-1.105763,13.354993,0.265851,0.000181
1,0,-0.043054,0.257463,-0.038333,0.0,0.0,-0.077638,-0.025940,0.067166,-0.038718,...,1.298854,0.000025,0.147961,13.547196,0.531931,0.000101,-0.935223,13.410417,0.375619,0.000104
2,0,-0.044468,0.271348,-0.038424,0.0,0.0,-0.078128,-0.025861,0.066498,-0.038869,...,1.407479,0.000023,0.226200,13.554549,0.600615,0.000097,-0.897425,13.424186,0.515443,0.000101
3,0,-0.036648,0.263665,-0.037134,0.0,0.0,-0.078205,-0.025329,0.067124,-0.038388,...,1.878035,0.000021,0.147232,13.555276,0.688513,0.000091,-0.963557,13.446701,0.967534,0.000093
4,0,-0.055866,0.249767,-0.037165,0.0,0.0,-0.078036,-0.026659,0.063233,-0.038521,...,1.770774,0.000021,0.125505,13.488442,0.709922,0.000088,-0.971129,13.372814,0.771981,0.000092


In [3]:
# drop rows with NaN (incomplete captures)
df = df.dropna()

label_col = 'label'

X = df.drop(columns=label_col).values.astype('float32')
y = df[label_col].values.astype('int')

print('X shape raw:', X.shape)

#shuffle split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, stratify=y, random_state=42)

# normalize features (StandardScaler)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print(X_train)

print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

# Conv1D expects shape (samples, steps, channels)
# We'll treat features as "time steps" and channels=1
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

print('X_train shape (for Conv1D):', X_train.shape)

X shape raw: (40, 1566)
[[ 0.14737955  1.8102282  -1.2084067  ...  0.82186157  1.2369373
  -0.950876  ]
 [ 0.26657978  0.5586724  -0.17574625 ...  1.0870976   1.2846467
   0.24278744]
 [-1.3025548   0.772099   -0.44404864 ...  0.3355316  -1.5838807
   4.137683  ]
 ...
 [-0.7655692   0.3150655  -0.46396044 ...  1.1788912   0.6586177
  -0.95122576]
 [-2.0696664  -0.1437988  -0.6729154  ...  0.30931047  1.340047
  -0.19022992]
 [ 0.13468653  1.5114791   0.18029703 ...  0.36818495 -0.87653935
  -1.1292373 ]]
X_train shape: (34, 1566)
X_test shape: (6, 1566)
X_train shape (for Conv1D): (34, 1566, 1)


In [4]:
def make_model(input_shape):
    model = models.Sequential()
    model.add(layers.Input(shape=input_shape))
    model.add(layers.Conv1D(64, kernel_size=3, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv1D(128, kernel_size=3, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.GlobalAveragePooling1D())
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = make_model((X_train.shape[1], 1))
model.summary()

2025-10-28 11:07:32.434930: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 1564, 64)       │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1564, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 1562, 128)      │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 1562, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 34,049 (133.00 KB)

 Trainable params: 33,665 (131.50 KB)

 Non-trainable params: 384 (1.50 KB)

In [ ]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)
]

history = model.fit(X_train, y_train, validation_split=0.15, epochs=80, batch_size=32, callbacks=callbacks)

# plot training
plt.figure()
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend()
plt.show()

In [ ]:
loss, acc = model.evaluate(X_test, y_test)
print('test loss', loss, 'test acc', acc)

model.save('drowsy_cnn_model.keras')

# also save scaler
import joblib
joblib.dump(scaler, 'scaler.save')